In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


In [36]:
comments = pd.read_csv('UScomments.csv', on_bad_lines='skip')


In [37]:
videos = pd.read_csv('USvideos.csv', on_bad_lines='skip')

In [38]:
comments.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [39]:
videos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [40]:
comments.nunique()

video_id          2266
comment_text    434076
likes             1284
replies            479
dtype: int64

In [41]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691400 entries, 0 to 691399
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   video_id      691400 non-null  object
 1   comment_text  691375 non-null  object
 2   likes         691400 non-null  object
 3   replies       691400 non-null  object
dtypes: object(4)
memory usage: 21.1+ MB


In [42]:
comments.isnull().sum()

video_id         0
comment_text    25
likes            0
replies          0
dtype: int64

In [43]:
comments.dropna(inplace=True)

In [44]:
comments.loc[41587]

video_id         _ANP3HR1jsM
comment_text    comment_text
likes                  likes
replies              replies
Name: 41587, dtype: object

In [45]:
comments.drop(41587, inplace=True)

In [46]:
comments = comments.reset_index().drop('index',axis=1)

In [47]:
comments.likes = comments.likes.astype(int)
comments.replies = comments.replies.astype(int)

In [48]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691374 entries, 0 to 691373
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   video_id      691374 non-null  object
 1   comment_text  691374 non-null  object
 2   likes         691374 non-null  int32 
 3   replies       691374 non-null  int32 
dtypes: int32(2), object(2)
memory usage: 15.8+ MB


In [49]:
videos.nunique()

video_id          2364
title             2398
channel_title     1230
category_id         16
tags              2204
views             7939
likes             6624
dislikes          2531
comment_total     4152
thumbnail_link    2364
date                40
dtype: int64

In [50]:
comments['comment_text'] = comments['comment_text'].str.replace("[^a-zA-Z#]", " ")

In [51]:
comments['comment_text'] = comments['comment_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [52]:
comments['comment_text'] = comments['comment_text'].apply(lambda x:x.lower())

In [53]:
tokenized_tweet = comments['comment_text'].apply(lambda x: x.split())
tokenized_tweet.head()

0                                        [logan, paul]
1    [been, following, from, start, your, vine, cha...
2                                     [kong, maverick]
3                                         [attendance]
4                                           [trending]
Name: comment_text, dtype: object

In [54]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [55]:
wnl = WordNetLemmatizer()

In [56]:
tokenized_tweet.apply(lambda x: [wnl.lemmatize(i) for i in x if i not in set(stopwords.words('english'))]) 
tokenized_tweet.head()

0                                        [logan, paul]
1    [been, following, from, start, your, vine, cha...
2                                     [kong, maverick]
3                                         [attendance]
4                                           [trending]
Name: comment_text, dtype: object

In [57]:
a = tokenized_tweet.copy()

In [58]:
a.head()

0                                        [logan, paul]
1    [been, following, from, start, your, vine, cha...
2                                     [kong, maverick]
3                                         [attendance]
4                                           [trending]
Name: comment_text, dtype: object

In [59]:
for i in range(len(a)):
    a[i] = ' '.join(a[i])

In [60]:
comments['token_text'] = a

In [61]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [62]:
comments['Sentiment Scores'] = comments['token_text'].apply(lambda x:sia.polarity_scores(x)['compound'])

In [63]:
comments.head()

,video_id,comment_text,likes,replies,token_text,Sentiment Scores
0,XpVt6Z1Gjjo,logan paul,4,0,logan paul,0.0
1,XpVt6Z1Gjjo,been following from start your vine channel ha...,3,0,been following from start your vine channel ha...,0.0
2,XpVt6Z1Gjjo,kong maverick,3,0,kong maverick,0.0
3,XpVt6Z1Gjjo,attendance,3,0,attendance,0.0
4,XpVt6Z1Gjjo,trending,3,0,trending,0.0


In [64]:
comments['Sentiment'] = \
comments['Sentiment Scores'].apply(lambda s : 'Positive' if s > 0 else ('Neutral' if s == 0 else 'Negative'))

In [67]:
comments.head()

,video_id,comment_text,likes,replies,token_text,Sentiment Scores,Sentiment
0,XpVt6Z1Gjjo,logan paul,4,0,logan paul,0.0,Neutral
1,XpVt6Z1Gjjo,been following from start your vine channel ha...,3,0,been following from start your vine channel ha...,0.0,Neutral
2,XpVt6Z1Gjjo,kong maverick,3,0,kong maverick,0.0,Neutral
3,XpVt6Z1Gjjo,attendance,3,0,attendance,0.0,Neutral
4,XpVt6Z1Gjjo,trending,3,0,trending,0.0,Neutral


In [66]:
comments.Sentiment.value_counts()

Positive    305358
Neutral     260986
Negative    125030
Name: Sentiment, dtype: int64

In [141]:
len(comments)

691374

In [145]:
channels = []
for i in range(len(comments)):
    channels.append(Positivity[Positivity.video_id == comments.video_id[i]]['Channel'].unique()[0])

In [146]:
comments['channel'] = channels

In [164]:
comments.head()

,video_id,comment_text,likes,replies,token_text,Sentiment Scores,Sentiment,channel
0,XpVt6Z1Gjjo,logan paul,4,0,logan paul,0.0,Neutral,Logan Paul Vlogs
1,XpVt6Z1Gjjo,been following from start your vine channel ha...,3,0,been following from start your vine channel ha...,0.0,Neutral,Logan Paul Vlogs
2,XpVt6Z1Gjjo,kong maverick,3,0,kong maverick,0.0,Neutral,Logan Paul Vlogs
3,XpVt6Z1Gjjo,attendance,3,0,attendance,0.0,Neutral,Logan Paul Vlogs
4,XpVt6Z1Gjjo,trending,3,0,trending,0.0,Neutral,Logan Paul Vlogs


In [ ]:
import dash # 建立dash
import dash_table # 要轉成資料框必用
from dash.dependencies import Input, Output # 在callback時使用
import dash_core_components as dcc # 製作Dashboard上的功能
import dash_html_components as html # 製作Dashboard網頁
import plotly.graph_objs as go # 畫各種圖
import plotly.express as px

In [188]:
app = dash.Dash(__name__)

app.layout = html.Div([
    
                html.Label('Youtube Sentiment Analysis'),
                dcc.Dropdown(
                    id='video_dropdown',
                    options=comments.channel.unique().tolist(),
                    placeholder='Search a Video',
                    clearable=False,
                    
                    value='Logan Paul Vlogs',
                    style = {'width':'45%'}
                ),
                
            dcc.Graph(
                    id='Pie-Chart'),
])       
    
    
@app.callback(Output('Pie-Chart', 'figure'),
            [Input('video_dropdown', 'value')])
    
def plot_pie_chart(video_dropdown):
    
    dff = comments.copy()
    piechart = px.pie(
                        data_frame = dff[dff['channel'] == video_dropdown],
                        names = 'Sentiment',
                        hole = .3,
                        )
    
    return piechart
    
    
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2022 23:24:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 23:24:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 23:24:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 23:24:17] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2022 23:24:17] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2022 23:24:17] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2022 23:24:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 23:24:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 23:24:22] "POST /_dash-update-component HTTP/1.1" 200 -


In [68]:
percentage = pd.DataFrame(comments  ,comments.video_id.unique()).reset_index()

In [73]:
videos_id = []
for i in range(0,comments.video_id.nunique()):
    a = comments[(comments.video_id == comments.video_id.unique()[i]) & (comments.Sentiment == 'Positive')].count()[0]
    b = comments[comments.video_id == comments.video_id.unique()[i]]['Sentiment'].value_counts().sum()
    Percentage = (a/b)*100
    videos_id.append(round(Percentage,2))

In [74]:
Positivity = pd.DataFrame(videos_id,comments.video_id.unique()).reset_index()


In [75]:
Positivity.columns = ['video_id','Positive Percentage']

In [76]:
Positivity.head()

,video_id,Positive Percentage
0,XpVt6Z1Gjjo,34.38
1,cLdxuaxaQwc,35.54
2,WYYvHb03Eog,37.25
3,sjlHnJvXdQs,37.62
4,cMKX2tE5Luk,41.57


In [77]:
channels = []
for i in range(0,Positivity.video_id.nunique()):
    channels.append(videos[videos.video_id == Positivity.video_id.unique()[i]]['channel_title'].unique()[0])

In [78]:
Positivity['Channel'] = channels

In [79]:
Positivity.head()

,video_id,Positive Percentage,Channel
0,XpVt6Z1Gjjo,34.38,Logan Paul Vlogs
1,cLdxuaxaQwc,35.54,PewDiePie
2,WYYvHb03Eog,37.25,The Verge
3,sjlHnJvXdQs,37.62,jacksfilms
4,cMKX2tE5Luk,41.57,A24


In [80]:
Positivity[Positivity['Positive Percentage'] == Positivity['Positive Percentage'].max()]

,video_id,Positive Percentage,Channel
395,cviyIIvBlto,100.0,Kerry Wong
424,jLa9wZHiqxg,100.0,Rad Universe
466,S-a8NVEUO4E,100.0,E! Live from the Red Carpet
601,wBjAmThxDpA,100.0,Vertical Entertainment LA
612,_zl2GV89_GM,100.0,Cosmic Book News
821,lJj66BCiZWQ,100.0,Eric Blattberg
972,BNRUV3TuExQ,100.0,Fathom Events
1317,zuKX0fPlo2Q,100.0,Thomas Bikias
1454,Ekc7lWx0468,100.0,The Illusion contest
1539,HjfN2Phsfng,100.0,weatherguru76


In [81]:
Positivity[Positivity['Positive Percentage'] == Positivity['Positive Percentage'].min()]


,video_id,Positive Percentage,Channel
117,3hq2c9vdfls,0.0,Cal Bears
151,7Ag2oDIeLvo,0.0,OsbornTramain
420,p7sooI96zPI,0.0,Geezus66
468,Oz88_p125uM,0.0,ViralVideoUK
592,BfxOimRxXvU,0.0,Andro Hack
604,k8LF_FU2luY,0.0,Bollinger Motors
611,tbDr_zAcM5g,0.0,Maharishi University of Management
629,HpxsQ1_UAec,0.0,Manto kay SO Afsanay
811,EZaPeQ2dxh8,0.0,Malhar Takle
819,D6zUj1tKxiU,0.0,Xposure 365 TV
